In [1]:
import sqlite3
from decouple import config
import pandas as pd

In [2]:
# Create Env vars
DATA_SQL_PATH = config("DATA_SQL_PATH")

#### Criando o banco de dados SQLITE

In [3]:
engine = sqlite3.connect(DATA_SQL_PATH+"db/eebii-culminancy.db")
# Create a cursor
cursor = engine.cursor()

#### 1 - Carregar os dados dos arquivos nas respectivas tabelas 

In [4]:
# Função para fazer o load dos arquivos
def load_data_fromFile_txt (file_path:str) :

    with open(file_path, mode="r") as file :
        data = file.readlines()

    data = [line.strip().split(",") for line in data]

    return data

* Carregando os dados dos txt em variáveis

In [5]:
# Load course data
courseData = load_data_fromFile_txt(DATA_SQL_PATH+"SampleCourseData.txt")
# Load Credit data
creditData = load_data_fromFile_txt(DATA_SQL_PATH+"SampleCreditData.txt")
# Load Person data
personData = load_data_fromFile_txt(DATA_SQL_PATH+"SamplePersonData.txt")
# Load Student data
studentData = load_data_fromFile_txt(DATA_SQL_PATH+"SampleStudentData.txt")

#### Pre Process Data

In [6]:
# Pre process data COURSE
for line in courseData :
    line[0] = int(line[0])
# Pre process data CREDIT
for line in creditData :
    line[0] = int(line[0])
    line[1] = int(line[1])
    line[2] = float(line[2])
    line[3] = int(line[3])
# Pre process data PERSON
for line in personData :
    line[0] = int(line[0])
# Pre process data PERSON
for line in studentData :
    line[0] = int(line[0])
    line[1] = int(line[1])

* Criando os comandos de criação das tabelas

In [7]:
# Query to create table COURSE
courseDataQuery = '''
    CREATE TABLE IF NOT EXISTS course (
    courseId INTEGER PRIMARY KEY,
    name TEXT,
    teacher TEXT
    );'''
# Query to create table CREDIT
creditDataQuery = '''
    CREATE TABLE IF NOT EXISTS credit (
    courseId INTEGER,
    studentId INTEGER,
    grade REAL,
    attempt INTEGER,
    FOREIGN KEY (courseId) REFERENCES course (courseId)
    FOREIGN KEY (studentId) REFERENCES student (studentId)
    );'''
# Query to create table PERSON
personDataQuery = '''
    CREATE TABLE IF NOT EXISTS person (
    personId INTEGER PRIMARY KEY,
    firstName TEXT,
    middleInitial TEXT,
    lastName TEXT,
    dateOfBirth DATE
    );'''
# Query to create table STUDENT
studentDataQuery = '''
    CREATE TABLE IF NOT EXISTS student (
    studentId INTEGER PRIMARY KEY,
    personId INTEGER,
    email TEXT,
    FOREIGN KEY (personId) REFERENCES person (personId)
    );'''

* Execute creation of COURSE table

In [8]:
cursor.execute(courseDataQuery)
cursor.execute("PRAGMA table_info(course)").fetchall()

[(0, 'courseId', 'INTEGER', 0, None, 1),
 (1, 'name', 'TEXT', 0, None, 0),
 (2, 'teacher', 'TEXT', 0, None, 0)]

* Execute creation of CREDIT table

In [9]:
cursor.execute(creditDataQuery)
cursor.execute("PRAGMA table_info(credit)").fetchall()

[(0, 'courseId', 'INTEGER', 0, None, 0),
 (1, 'studentId', 'INTEGER', 0, None, 0),
 (2, 'grade', 'REAL', 0, None, 0),
 (3, 'attempt', 'INTEGER', 0, None, 0)]

* Execute creation of PERSON table

In [10]:
cursor.execute(personDataQuery)
cursor.execute("PRAGMA table_info(person)").fetchall()

[(0, 'personId', 'INTEGER', 0, None, 1),
 (1, 'firstName', 'TEXT', 0, None, 0),
 (2, 'middleInitial', 'TEXT', 0, None, 0),
 (3, 'lastName', 'TEXT', 0, None, 0),
 (4, 'dateOfBirth', 'DATE', 0, None, 0)]

* Execute creation of STUDENT table

In [11]:
cursor.execute(studentDataQuery)
cursor.execute("PRAGMA table_info(student)").fetchall()

[(0, 'studentId', 'INTEGER', 0, None, 1),
 (1, 'personId', 'INTEGER', 0, None, 0),
 (2, 'email', 'TEXT', 0, None, 0)]

#### 1.1 - Populando as tabelas

In [12]:
def populateTables (data:list, table:str, cursor:sqlite3.Cursor) : 
    
    table_schema = cursor.execute(f"PRAGMA table_info({table})").fetchall()
    table_schema = tuple([str(elem[1]) for elem in table_schema])
    print(f"Columns : {table_schema}")
    for line in data :
        line = tuple(line)
        
        cursor.execute(
            f'''
                INSERT INTO {table} {table_schema} VALUES {line};
            '''
        )
    
    verify_add = cursor.execute(
        f"SELECT * FROM {table}"
    ).fetchall()

    # return verify_add

In [13]:
# Populate COURSE table
populateTables(data=courseData, table='course', cursor=cursor)
# Populate CREDIT table
populateTables(data=creditData, table='credit', cursor=cursor)
# Populate PERSON table
populateTables(data=personData, table='person', cursor=cursor)
# Populate STUDENT table
populateTables(data=studentData, table='student', cursor=cursor)

Columns : ('courseId', 'name', 'teacher')
Columns : ('courseId', 'studentId', 'grade', 'attempt')
Columns : ('personId', 'firstName', 'middleInitial', 'lastName', 'dateOfBirth')
Columns : ('studentId', 'personId', 'email')


* Efetuando a busca por todos os registros da tabela Pessoas

In [14]:
query = "SELECT * FROM person"
df = pd.read_sql_query(query, engine)

display(df)

,personId,firstName,middleInitial,lastName,dateOfBirth
0,1,Adele,S,Rosales,1984-05-05
1,2,Justus,Z,Gomez,2007-04-06
2,3,Isiah,Z,Warner,1992-06-04
3,4,Erica,L,Campos,1994-05-14
4,5,Desmond,W,Valdez,1996-11-19
...,...,...,...,...,...
995,996,Piper,D,Dunn,1980-07-20
996,997,Emmalyn,Y,Barr,1983-09-07
997,998,Audrina,X,Solis,1994-02-24
998,999,Tyler,I,Harrell,1992-02-17


* Efetuando a busca por todos os registros da tabela Pessoas que tenham sobrenome começados em "H"

In [18]:
query = "SELECT * FROM person WHERE lastName LIKE 'H%'"
df = pd.read_sql_query(query, engine)

display(df)

,personId,firstName,middleInitial,lastName,dateOfBirth
0,9,Haleigh,M,Hopper,1997-07-24
1,19,Amare,A,Hernandez,2000-09-17
2,35,Laura,G,Herrera,1988-05-12
3,36,Lyric,S,Horton,2008-10-24
4,37,Deandre,F,Hopper,2007-03-31
...,...,...,...,...,...
88,963,Reyna,C,Hinton,2002-07-03
89,967,Averi,D,Holman,2003-04-26
90,975,Bianca,V,Hancock,1988-07-14
91,987,Teagan,Y,Hendrix,1989-10-26


* Consulta com condição de Primeiro Nome e SobreNome comecem com a letra 'B'

In [16]:
query = "SELECT * FROM person WHERE lastName LIKE 'B%' AND firstName LIKE 'B%'"
df = pd.read_sql_query(query, engine)

df

,personId,firstName,middleInitial,lastName,dateOfBirth
0,447,Bodhi,J,Best,1996-07-06
1,571,Bruno,E,Bolton,2008-04-02
2,670,Brock,X,Barr,1987-11-12


* Consulta com condição de data de nascimento ser maior que '01/01/2000'

In [17]:
query = '''
    SELECT  * FROM person WHERE dateOfBirth > '2000-01-01'
'''
df = pd.read_sql_query(query, engine)

df

,personId,firstName,middleInitial,lastName,dateOfBirth
0,2,Justus,Z,Gomez,2007-04-06
1,6,Arnav,R,Powers,2008-05-12
2,11,Jacob,P,Dotson,2008-01-02
3,12,Aden,F,Petty,2007-12-02
4,15,Karma,N,Guerrero,2005-02-18
...,...,...,...,...,...
318,988,Campbell,T,Chase,2006-07-11
319,989,Lexie,S,Rich,2008-04-25
320,990,Reagan,S,Barton,2005-07-01
321,993,Courtney,N,Dale,2003-02-25
